# Export Computer Vision Models on Torchvision via PyTorch

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://drive.google.com/file/d/17QlUHJv6ekWPY7v8Jg6dW-7tPjiuNZ1N/view?usp=sharing">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
</table>
<br>
</table>


> This script is recommended to be run on **Google Colab** or a **Local Server**. If you are using your server, we also recommend that you set up a virtual environment via python-venv or conda before running this script, as it requires the specified version of the framework to run as expected.

In [1]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0
!pip install torchsummary

* Config the `model_name` and `torchvision.models.<backbone>` you want.

> **Note**: You can find all the available model options in the official  [Torchvision](https://pytorch.org/vision/stable/models.html) documentation.

### Get Pre-trained Weights and Summary of the Model's Configuration

Get a list of all models available through [Official PyTorch Documentation](https://pytorch.org/vision/stable/models.html).

In [2]:
import torch, torchvision, os

# Please set the input dimensions of the model and turn off the gradient mode to adjust it to run mode.
model_name = 'resnet18'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.resnet18(True).to(device).eval()  # Evaluation mode is necessary to fix the weight when export.
model_name, device

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


('resnet18', device(type='cpu'))

Specify the `input_size` of the model (usually follows the specifications of the dataset being used for pre-training).

In [3]:
from torchsummary import summary

inputs = torch.randn(1, 3, 224, 224, requires_grad=False).to(device)
summary(model, inputs.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [4]:
# Verify the Output Value. You can also use PyTorch API to train the model.
OUTPUT_TORCHSCRIPT = model(inputs).detach().numpy()
OUTPUT_TORCHSCRIPT.shape

(1, 1000)

### Convert the Torch model into ONNX format

> **Note**: Please check which OPS version of ONNX is supported by the accelerator before you converting to TFLite.

In [5]:
!pip install onnxruntime
!pip install onnx

Verify the consistency of the model output. If the result is `True` then back up the model to [Azure Instance (itriaihub)]() and share it to [AI-Hub](https://github.com/R300-AI/ITRI-AI-Hub/tree/main).

In [6]:
import numpy as np
import onnxruntime

torch.onnx.export(model, inputs, f"{model_name}.onnx", verbose=False, opset_version=11, do_constant_folding=False, dynamic_axes=None)
ort_session = onnxruntime.InferenceSession(f'{model_name}.onnx')
OUTPUT_ONNX = ort_session.run([ort_session.get_outputs()[0].name], {ort_session.get_inputs()[0].name: inputs.numpy()})[0]

print('Model Consistancy Check Passed (ONNX):', np.allclose(OUTPUT_TORCHSCRIPT, OUTPUT_ONNX, rtol=1e-3, atol=1e-6))

Model Consistancy Check Passed (ONNX): True


Now we can save the model to a local server and visualise it using Netron.

> **Netron** can either be installed on Macos, Linux or Windows server via `pip install netron` or run directly in [browser](https://netron.app/).

![image](https://pytorch.org/tutorials/_static/img/onnx/netron_web_ui.png).

### Convert the ONNX model into TFLite format

Before converting to TFLite, check which OPS version of TFLite is supported by the accelerator.


In [7]:
!pip install keras
!pip install tf_keras
!pip install "sng4onnx>=1.0.1
!pip install "onnx_graphsurgeon>=0.3.26"
!pip install "onnx2tf>1.17.5,<=1.22.3"
!pip install "onnxslim>=0.1.31"
!pip install sng4onnx

!rm -rf {model_name}
os.mkdir(model_name)

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [8]:
import onnx2tf
import tensorflow as tf

onnx2tf.convert(
    input_onnx_file_path=f"{model_name}.onnx",
    output_folder_path=model_name,
    not_use_onnxsim=True, non_verbose=False, verbosity=1,
    copy_onnx_input_output_names_to_tflite=True,
    output_integer_quantized_tflite=False,
    quant_type="per-tensor",  # "per-tensor" (faster) or "per-channel" (slower but more accurate)
)

print(f"Input shape: {tf.lite.Interpreter(model_path=f'{model_name}/{model_name}_float32.tflite').get_input_details()[0]['shape']}")


Automatic generation of each OP name started ========================================
Automatic generation of each OP name complete!

Model loaded ========================================================================

Model conversion started ============================================================
saved_model output started ==========================================================
saved_model output complete!
wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float32 tflite output complete!
wget https://github.com/PINTO0309/onnx2tf/releases/download/1.16.31/flatc.tar.gz && tar -zxvf flatc.tar.gz && sudo chmod +x flatc && sudo mv flatc /usr/bin/
Float16 tflite output complete!
Input shape: [  1 224 224   3]


In [9]:
interpreter = tf.lite.Interpreter(model_path=f'{model_name}/{model_name}_float32.tflite')
interpreter.allocate_tensors()

interpreter.set_tensor(interpreter.get_input_details()[0]['index'], inputs.permute(0, 2, 3, 1).numpy())
interpreter.invoke()
OUTPUT_TFLITE = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])[0]

print('Model Conformance Check Passed (TFLite):', np.allclose(OUTPUT_TORCHSCRIPT, OUTPUT_ONNX, rtol=1e-3, atol=1e-6))

Model Conformance Check Passed (TFLite): True
